### Apriori

In [72]:
# load the data, and here we simply create some example data
def load_data(): 
    dataset = [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]
    return dataset

In [73]:
def createC1(dataset):
    C1 = []
    for row in dataset:  #scan each row
        for item in row: #scan each item in each row
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)

In [80]:
def scanD(dataset, candidate, minSupport):
    candidate_dict = {} # a dictionary which stores <candidate, frequency>
    
    # scan each candidate in the dataset and find the frequency
    for row in dataset: # each row
        for item in candidate: # each candidate
            if item.issubset(row): # subset checks row contains the candidate or not
                if not item in candidate_dict:
                    candidate_dict[item] = 1
                else:
                    candidate_dict[item] += 1
    
    numRows = float(len(dataset))
    validItemList = [] # valid candidate list
    supportData = {}
    
    # calculate support for each candidate and prune
    for key in candidate_dict:
        support = candidate_dict[key] / numRows
        if support >= minSupport:
            validItemList.insert(0,key) # add valid candidates into the list
        supportData[key] = support # save the support data
    
    return validItemList, supportData

In [75]:
# CandidateListK is the k-th list(table) of itemsets, k is the number of items in each row
def aprioriGenerate(CandidateListK, k):
    newList = []
    lenk = len(CandidateListK)
    for i in range(lenk): # scan each row in the candidate list
        for j in range(i + 1, lenk): # scan other rows
            L1 = list(CandidateListK[i])[:k-2]  
            L2 = list(CandidateListK[j])[:k-2]
            if L1 == L2:
                newList.append(CandidateListK[i] | CandidateListK[j])
    return newList

In [76]:
def apriori(dataset, minSupport = 0.5):  
    # from size 1
    C1 = list(createC1(load_data())) # frequent 1-item set
    dataset = list(map(set, load_data())) # whole data set
    L1, supportData = scanD(dataset, C1, minSupport)
    
    # list of candidates at each step
    L = [L1]
    k = 2
    # if the length of itemset at k-th step is k-1, then the first k-2 items should be the same
    while(len(L[k-2]) > 0):
        CandidateListK = aprioriGenerate(L[k-2],k)
        Lk, supportK = scanD(dataset, CandidateListK, minSupport)
        supportData.update(supportK)
        L.append(Lk)
        k += 1
    return L, supportData

In [89]:
# run
L, supportData = apriori(dataset)
i = 0
for j in L:
    print ("frequency of: {}".format(i + 1), j , "\n")
    i +=1

frequency of: 1 [frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})] 

frequency of: 2 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})] 

frequency of: 3 [frozenset({2, 3, 5})] 

frequency of: 4 [] 



In [6]:
# There is an existring library which implements the Apriori algorithm
from efficient_apriori import apriori

transactions = [('eggs', 'bacon', 'soup', 'cheese'),
                ('eggs', 'bacon', 'apple', 'cheese'),
                ('soup', 'bacon', 'banana', 'milk')]

itemsets, rules = apriori(transactions, min_support=0.5,  min_confidence=1)

print(rules) 

[{cheese} -> {bacon}, {eggs} -> {bacon}, {soup} -> {bacon}, {eggs} -> {cheese}, {cheese} -> {eggs}, {cheese, eggs} -> {bacon}, {bacon, eggs} -> {cheese}, {bacon, cheese} -> {eggs}, {eggs} -> {bacon, cheese}, {cheese} -> {bacon, eggs}]


### FP-growth

In [1]:
import pyfpgrowth

In [2]:
transactions = [[1, 2, 5],
                [2, 4],
                [2, 3],
                [1, 2, 4],
                [1, 3],
                [2, 3],
                [1, 3],
                [1, 2, 3, 5],
                [1, 2, 3]]

In [3]:
# threshold : 2
patterns = pyfpgrowth.find_frequent_patterns(transactions, 2)

In [6]:
# minimum support: 0.7
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)
print(rules)

{(5,): ((1, 2), 1.0), (1, 5): ((2,), 1.0), (2, 5): ((1,), 1.0), (4,): ((2,), 1.0)}
